In [1]:
import pandas as pd
import numpy as np
import time
import pickle
from math import floor
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import re
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
import nltk

In [ ]:
def get_titles_and_urls(website_list):
    '''To acquire a list of titles and website extenstions for gutenberg.org from the Bookshelves part of the website. 
    Requires a list of website urls to be passed in. 
    Returns titles and link extensions'''
    titles = []
    links = []
    for counter, website in enumerate(website_list):
        driver = webdriver.Firefox()
        driver.implicitly_wait(5)
        driver.get(website)
        for x, y  in enumerate(driver.find_elements_by_xpath('./html/body/div[1]/div[2]/div[2]/div/ul/li')):
            try:
                if [i.get_attribute('alt') for i in y.find_elements_by_xpath('./a[@class="image"]/img')][0] == 'BookIcon.png':
                    titles.append(y.find_elements_by_xpath('./a')[0].text)
                    links.append(y.find_elements_by_xpath('./a')[0].get_attribute('href'))
                else:
                    continue
            except:
                continue
        driver.quit()
        print('Website', counter+1, 'information acquired!')
    print('Is length of titles scrapped and links scrapped equal?', len(titles)==len(links))
    return titles, links

In [ ]:
websites = ["https://www.gutenberg.org/wiki/Children%27s_Literature_(Bookshelf)", 
            "https://www.gutenberg.org/wiki/Children%27s_Fiction_(Bookshelf)",
            "https://www.gutenberg.org/wiki/Adventure_(Bookshelf)",
            "https://www.gutenberg.org/wiki/Fantasy_(Bookshelf)",
            "https://www.gutenberg.org/wiki/Humor_(Bookshelf)",
            "https://www.gutenberg.org/wiki/Mystery_Fiction_(Bookshelf)",
            "https://www.gutenberg.org/wiki/Movie_Books_(Bookshelf)",
            "https://www.gutenberg.org/wiki/Science_Fiction_(Bookshelf)",
            "https://www.gutenberg.org/wiki/Children%27s_History_(Bookshelf)",
            "https://www.gutenberg.org/wiki/Plays_(Bookshelf)"]
titles, links = get_titles_and_urls(websites)

In [ ]:
links_to_search = []
for x in links:
    links_to_search.append(x.split('/')[4])
len(links_to_search)

In [ ]:
texts = []
urls_used = []
urls_not_used = []
for i in range(0, len(links_to_search)):
    to_find = links_to_search[i]
    print(i)
    try:
        driver = webdriver.Firefox()
        driver.implicitly_wait(5)
        driver.get("https://www.gutenberg.org/ebooks/"+to_find+".txt.utf-8")
        text = driver.find_element_by_xpath('//pre')
        texts.append(text.text)
        driver.quit()
        urls_used.append(i)
        print('Text acquired!')
    except NoSuchElementException as inst:
        print(inst)          # __str__ allows args to be printed directly,
        driver = webdriver.Firefox()
        driver.implicitly_wait(5)
        driver.get("https://www.gutenberg.org/files/"+to_find+"/"+to_find+"-0.txt")
        text = driver.find_element_by_xpath('//pre')
        texts.append(text.text)
        driver.quit()
        urls_used.append(i)
        print('Text acquired!')
    driver.quit()

In [ ]:
len(texts)

In [ ]:
file = open('full_texts.pickle', 'wb')

In [ ]:
pickle.dump([titles, texts], file)

In [ ]:
file.close()

In [ ]:
file = open('full_texts.pickle', 'rb')

In [ ]:
full_text_titles, full_text_texts = pickle.load(file)

In [ ]:
special_cases = [57, 158, 189, 189]
for x in special_cases:
    full_text_titles.pop(x)
    full_text_texts.pop(x)

In [ ]:
driver = webdriver.Firefox()
driver.implicitly_wait(5)
driver.get("https://hub.lexile.com/find-a-book/search")
analytics = driver.find_element_by_xpath('//label[@for="checkbox_1"]')
result = analytics.is_selected()
if result:
    print('Checkbox already selected')
else:
    analytics.click()
    print('Analytics Checkbox deselected.')
save_analytics_button = driver.find_element_by_xpath('//button[text()="Save Preferences"]')
save_analytics_button.click()

In [ ]:
books = []
authors = []
lexiles = []

In [ ]:
for x in range(0, len(full_text_titles)):
    print(x)
    value = str(full_text_titles[x]).replace("/", ' ')
    search_bar = driver.find_element_by_xpath('//input[@name="quickSearch"]')
    search_bar.send_keys(value)
    search_button = driver.find_element_by_xpath('//button[text()="Search"]')
    search_button.click()
    print('Finding books')
    titles = driver.find_elements_by_xpath('//*[@data-testid="book-title"]')
    book_titles = []
    for result in titles:
        book_titles.append(result.text)
    print('Finding authors')
    authors_of_books = driver.find_elements_by_xpath('//*[@data-testid="book-authors"]')
    book_authors = []
    for result in authors_of_books:
        book_authors.append(result.text)
    print('Finding levels')
    levels = driver.find_elements_by_class_name('sc-iFMziU.ixIPnd')
    lexile_levels = []
    for result in levels:
        lexile_levels.append(result.text)
    books.append(book_titles)
    authors.append(book_authors)
    lexiles.append(lexile_levels)
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[text()="New Search"]')))
    element.click()
driver.close()

In [ ]:
file.close()

In [ ]:
file = open('lexile_levels.pickle', 'wb')

In [ ]:
pickle.dump([books, authors, lexiles], file)

In [ ]:
file.close()

In [ ]:
parse1 = []
parse2 = []
parse3 = []
parse4 = []
titles_from_text = []
for x in range(0, len(full_text_texts)):
#     print('1. removing license agreement at end')
    parse1.append(re.sub('\n', ' ', full_text_texts[x].splitlines()))
#     print('2. removing extras at the end of the text')
    if len(parse1[x].split('THE END'))==2:
        parse2.append(parse1[x].split('THE END')[0])
    elif len(parse1[x].split("End of the Project Gutenberg EBook"))==2:
        parse2.append(parse1[x].split("End of the Project Gutenberg EBook")[0])
    elif len(parse1[x].split('The End'))==2:
        parse2.append(parse1[x].split('The End')[0])
    elif len(parse1[x].split('End of Project Gutenberg\'s'))==2:
        parse2.append(parse1[x].split('End of Project Gutenberg\'s')[0])
    elif len(parse1[x].split('END OF THE PROJECT GUTENBERG EBOOK'))==2:
        parse2.append(parse1[x].split('END OF THE PROJECT GUTENBERG EBOOK')[0]) 
    elif len(parse1[x].split('END OF THIS PROJECT GUTENBERG EBOOK'))==2:
        parse2.append(parse1[x].split('END OF THIS PROJECT GUTENBERG EBOOK')[0])
    elif len(parse1[x].split('The Project Gutenberg Etext'))==4:
        parse2.append(parse1[x].split('The Project Gutenberg Etext')[2])
    elif len(parse1[x].split('*Project Gutenberg Etext'))==4:
        parse2.append(parse1[x].split('*Project Gutenberg Etext')[2])
    else:
        print('problem at: ', x)
#     print('3. removing anything at ebook related at beginning of text')
    if len(parse2[x].split('START OF THIS PROJECT GUTENBERG EBOOK'))==2:
        parse3.append(parse2[x].split('START OF THIS PROJECT GUTENBERG EBOOK')[1])
    elif len(parse2[x].split("START OF THE PROJECT GUTENBERG EBOOK"))==2:
        parse3.append(parse2[x].split("START OF THE PROJECT GUTENBERG EBOOK")[1])
    elif len(parse2[x].split("THIS EBOOK WAS ONE OF PROJECT GUTENBERG'S EARLY FILES PRODUCED AT A TIME WHEN PROOFING METHODS AND TOOLS WERE NOT WELL DEVELOPED."))==2:
        parse3.append(parse2[x].split("THIS EBOOK WAS ONE OF PROJECT GUTENBERG'S EARLY FILES PRODUCED AT A TIME WHEN PROOFING METHODS AND TOOLS WERE NOT WELL DEVELOPED.")[1])
    elif len(parse2[x].split("THE SMALL PRINT! FOR PUBLIC DOMAIN ETEXTS"))==2:
        parse3.append(parse2[x].split("THE SMALL PRINT! FOR PUBLIC DOMAIN ETEXTS")[1].split('*END*')[1])
    else:
        print('problem at: ', x)
    if len(parse3[x].split('***'))==2:
        parse4.append(parse3[x].split('***')[1])
    elif len(parse3[x].split('***'))==48:
        parse4.append(parse3[x].split('***')[47].split('*')[1])
    elif len(parse3[x].split('***'))==3:
        if len(parse3[x].split('***')[1].split('Online Distributed Proofreading Team'))==2:
            parse4.append(parse3[x].split('***')[1].split('Online Distributed Proofreading Team')[1])
        elif len(parse3[x].split('***')[1].split('Online Distributed Proofreading Team'))==1:
            if len(parse3[x].split('***')[1].split('Online Distributed Proofreading Team')[0].split('.zip)'))==2:
                parse4.append(parse3[x].split('***')[1].split('Online Distributed Proofreading Team')[0].split('.zip)')[1])
            elif len(parse3[x].split('***')[1].split('Online Distributed Proofreading Team')[0].split('.zip)'))==1:
                if len(parse3[x].split('***')[1].split('Online Distributed Proofreading Team')[0].split('.zip)')[0].split('.org'))==2:
                    parse4.append(parse3[x].split('***')[1].split('Online Distributed Proofreading Team')[0].split('.zip)')[0].split('.org')[1])
                elif len(parse3[x].split('***')[1].split('Online Distributed Proofreading Team')[0].split('.zip)')[0].split('.ac.uk'))==2:
                    parse4.append(parse3[x].split('***')[1].split('Online Distributed Proofreading Team')[0].split('.zip)')[0].split('.ac.uk')[1])
                elif len(parse3[x].split('***')[1].split('Online Distributed Proofreading Team')[0].split('.zip)')[0].split('.edu/women/.'))==2:
                    parse4.append(parse3[x].split('***')[1].split('Online Distributed Proofreading Team')[0].split('.zip)')[0].split('.edu/women/.')[0])
                elif len(parse3[x].split('***')[1].split('Online Distributed Proofreading Team')[0].split('.zip)')[0].split('     '))==839:
                    parse4.append(parse3[x].split('***')[1].split('Online Distributed Proofreading Team')[0].split('.zip)')[0].split('This eBook was')[1])
                else:
                    parse4.append(parse3[x])
            else:
                print('review: ', x)
        else:
            print('look at: ', x)
    elif len(parse3[x].split('***'))==7:
        parse4.append(parse3[x])
    else:
        print('problem at: ', x)
    print('Book number ', x, ' done!')
    titles_from_text.append(parse3[x].split('*** ')[0].strip().title())

In [ ]:
file = open('lexile_levels.pickle', 'rb')

In [ ]:
books, authors, lexiles = pickle.load(file)

In [ ]:
file.close()

In [ ]:
search_results_books = []
for i, j in enumerate(books):
    try:
        search_results_books.append(j[0].title())
    except:
        search_results_books.append('')
search_results_books

In [ ]:
search_results_lexiles = []
for i, j in enumerate(lexiles):
    try:
        search_results_lexiles.append(j[0])
    except:
        search_results_lexiles.append('')
search_results_lexiles

In [ ]:
length_lexiles = []
for x in search_results_lexiles:
    length_lexiles.append(x!='')
print('Number of scraped texts with associated lexile levels: ', sum(length_lexiles))

In [ ]:
labeled_data = pd.DataFrame()
labeled_lexiles = []
labeled_titles = []
labeled_text = []
for x in range(0, len(search_results_lexiles)):
    if str(search_results_books[x]) == str(titles_from_text[x]):
        labeled_lexiles.append(search_results_lexiles[x])
        labeled_titles.append(titles_from_text[x])
        labeled_text.append(parse4[x])
#         labels = list(zip(search_results_lexiles[x], titles_from_text[x]))
    else:
        print(x, 'No title or lexile match to scrapped material')
labeled_data['Lexiles'] = labeled_lexiles
labeled_data['Titles'] = labeled_titles
labeled_data['Texts'] = labeled_text
print('Number of scraped texts that can be labeled with lexile levels: ', len(labeled_data))
labeled_data.sample(5)